# 回転の推定I:等方性誤差

In [1]:
from pathlib import Path
import sys

import numpy as np
from scipy import linalg
from scipy.stats import special_ortho_group
from scipy.spatial.transform import Rotation

sys.path.append('../libs')
import util

In [2]:
A = util.load_point_cloud(Path('../bunny/data/bun180.ply').resolve())

In [ ]:
util.plot_3d(A)

In [3]:
R = special_ortho_group.rvs(3)
print(R)

[[-0.67160169  0.52410797 -0.52370031]
 [ 0.28109583 -0.47374539 -0.83459597]
 [-0.68551901 -0.70772604  0.17084364]]


In [4]:
noise = np.random.normal(0, 3e-3, A.shape)
A_prime = R @ A + noise

In [ ]:
util.plot_3d(A_prime)

## 4.3 特異値分解による解法

In [5]:
estimated_R = util.estimate_R_using_SVD(A, A_prime)
print(util.eval_R_error(estimated_R, R))

0.00046467862126025277


In [ ]:
util.plot_3d_multi(estimated_R @ A, A_prime)

## 4.4 四元数表示による解法

In [6]:
N = A @ A_prime.T
N_tilde = np.array([
    [
        N[0, 0] + N[1, 1] + N[2, 2],
        -N[2, 1] + N[1, 2],
        N[2, 0] - N[0, 2],
        -N[1, 0] + N[0, 1],
    ], [
        -N[2, 1] + N[1, 2],
        N[0, 0] - N[1, 1] - N[2, 2],
        N[1, 0] + N[0, 1],
        N[2, 0] + N[0, 2],
    ], [
        N[2, 0] - N[0, 2],
        N[1, 0] + N[0, 1],
        -N[0, 0] + N[1, 1] - N[2, 2],
        N[2, 1] + N[1, 2],
    ], [
        -N[1, 0] + N[0, 1],
        N[2, 0] + N[0, 2],
        N[2, 1] + N[1, 2],
        -N[0, 0] - N[1, 1] + N[2, 2],
    ],
])

In [7]:
w, v = linalg.eig(N_tilde)
max_vec = v[:, np.argmax(w)]
# 書籍では(w, x, y, z)だが，scipyでは(x, y, z, w)
r = Rotation.from_quat(max_vec[[1, 2, 3, 0]])
print(util.eval_R_error(r.as_dcm(), R))

0.00046467862126048603


In [ ]:
util.plot_3d_multi(r.as_dcm() @ A, A_prime)

## 4.5 回転行列の最適補正

In [8]:
R_hat = R + np.random.normal(0, 1e-2, R.shape)
R_hat @ R_hat.T

array([[ 1.00626274,  0.00372956, -0.00560683],
       [ 0.00372956,  0.98989348, -0.00938626],
       [-0.00560683, -0.00938626,  0.99968069]])

In [9]:
U, s, Vh = linalg.svd(R_hat)
V = Vh.T
corrected_R = U @ np.diag([1, 1, linalg.det(U @ V)]) @ Vh
corrected_R @ corrected_R.T

array([[ 1.00000000e+00, -2.21081875e-17, -1.17305016e-16],
       [-2.21081875e-17,  1.00000000e+00, -7.49677637e-16],
       [-1.17305016e-16, -7.49677637e-16,  1.00000000e+00]])